In [1]:
import pandas as pd
import numpy as np

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_4ee44b67c1364daa8980c509a11a2582 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_4ee44b67c1364daa8980c509a11a2582 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_4ee44b67c1364daa8980c509a11a2582 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='S74cX5nyL6zNO5GxQpfeG3AxM1bWDAOE7PYaIPBZUY6Y',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_4ee44b67c1364daa8980c509a11a2582)

body = client_4ee44b67c1364daa8980c509a11a2582.get_object(Bucket='croprecommendation-donotdelete-pr-pkrqpmzos7ngct',Key='cr_final.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
df.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            1200 non-null   int64  
 1   P            1200 non-null   int64  
 2   K            1200 non-null   int64  
 3   temperature  1200 non-null   float64
 4   humidity     1200 non-null   float64
 5   ph           1200 non-null   float64
 6   rainfall     1200 non-null   float64
 7   label        1200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 75.1+ KB


In [4]:
print("Missing Values")
for column in df.columns.tolist():
    print(column,":",df[column].isnull().sum())

Missing Values
N : 0
P : 0
K : 0
temperature : 0
humidity : 0
ph : 0
rainfall : 0
label : 0


In [5]:

df['label'].value_counts()

papaya       100
orange       100
rice         100
jute         100
grapes       100
apple        100
coffee       100
lentil       100
mango        100
blackgram    100
maize        100
banana       100
Name: label, dtype: int64

In [6]:
df.columns

Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label'], dtype='object')

In [7]:
features = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

In [8]:
X = df[features]
y = df[['label']]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.09, random_state=42)

In [10]:
print(X_train.shape)
print(X_test.shape)

(1092, 7)
(108, 7)


In [15]:
X_train

,N,P,K,temperature,humidity,ph,rainfall
1166,99,40,32,24.184712,69.948073,7.045543,163.270873
936,31,62,52,33.796016,93.007543,6.991041,182.026807
371,16,65,16,18.130278,62.458516,6.078724,50.612852
889,34,11,10,31.750489,94.595512,7.362208,115.198930
156,84,44,21,21.869274,61.910449,5.850440,107.268193
...,...,...,...,...,...,...,...
1044,94,37,41,24.763452,87.060711,6.463539,179.163086
1095,87,44,43,23.874845,86.792613,6.718725,177.514731
1130,98,27,27,24.713841,51.291425,7.238110,197.643971
860,31,25,12,18.051424,90.039696,7.016482,111.779389


In [16]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=110)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

<ipython-input-16-d07b156aea91>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,y_train)


In [17]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9907407407407407


In [18]:
feature_imp = pd.Series(clf.feature_importances_,index=['N','P','K','temperature','humidity','ph',"rainfall"]).sort_values(ascending=False)
feature_imp

rainfall       0.224303
K              0.198042
humidity       0.184942
P              0.166141
N              0.120412
temperature    0.056793
ph             0.049366
dtype: float64

In [19]:
from ibm_watson_machine_learning import APIClient

In [20]:
wml_credential={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"3vJ4WahpXCOtxmOvW4emBWPYpqNPaHDHu0mCnYv0aX3v"
}
client=APIClient(wml_credential)

In [21]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
#     print(space)
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [22]:
space_uid=guid_from_space_name(client,'models')
print("Space UID = "+space_uid)

Space UID = 20149eec-b8fe-4972-aa7a-8f624c2d77f9


In [23]:
client.set.default_space(space_uid)

'SUCCESS'

In [24]:
software_spec_uid=client.software_specifications.get_uid_by_name('default_py3.8')
software_spec_uid

'ab9e1b80-f2ce-592c-a7d2-4f2344f77194'

In [25]:
model_details=client.repository.store_model(model=clf,meta_props={
    client.repository.ModelMetaNames.NAME:"crop_reco",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_0.23",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

In [26]:
model_id=client.repository.get_model_uid(model_details)

In [27]:
model_id

'b472f2a3-ff66-4348-a0b7-3df0a9dcfd9f'